In [ ]:
import os
import glob
import time
import csv
import datetime
    
os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')
 
base_dir = '/sys/bus/w1/devices/'
device_folder = glob.glob(base_dir + '28*')[0]
device_file = device_folder + '/w1_slave'

#initialize the csv file

f = open("values.csv", "w") # overwrite any existing file
f.write("time,celcius,farenheit\n")
f.close()


def read_temp_raw():
    f = open(device_file, 'r')
    lines = f.readlines()
    f.close()
    return lines

def read_temp():
    lines = read_temp_raw()
    while lines[0].strip()[-3:] != 'YES':
        time.sleep(0.2)
        lines = read_temp_raw()
    equals_pos = lines[1].find('t=')
    if equals_pos != -1:
        temp_string = lines[1][equals_pos+2:]
        temp_c = float(temp_string) / 1000.0
        temp_f = temp_c * 9.0 / 5.0 + 32.0
        return temp_c, temp_f
    
while True:
    temps = read_temp()
    f = open("values.csv", "a") #append to file
    f.write("{0},{1},{2}\n".format(datetime.datetime.now(), temps[0], temps[1]))
    f.close()
    sleep(1)

In [ ]:
import csv
from flask import Flask
from flask import send_file
from flask import request
import subprocess
import sys
import datetime

# Starts the script

pid = subprocess.Popen([sys.executable, "sensors.py"])

# Flask Server

app = Flask(__name__)

@app.route('/')
def hello_world():
    return """
    <html>
  <head>
    <title>Aedea's Web Server</title>
  </head>
  <body>
    <h1>Current time: {0}</h1>
    <p>Reload this page when you start and copy the time into a document.</p>
    <br><br>
    <a href="/get">Download CSV</a>
    <br />
    <h4>Directions</h4>
    <p>
      1. Plug in Raspberry Pi, this will start collecting data ASAP<br />2.
      Record time<br />3. Perform experiment<br />4. When done, press "Download
      CSV" and record the time<br />5. Get rid of any data that's outside the
      recorded start and stop times.
    </p>
  </body>
</html>
""".format(datetime.datetime.now())

@app.route('/get')
def return_files_tut():
#     startDate = datetime.datetime.strptime(request.args.get('start'), '%Y-%m-%d %H:%M:%S.%f')
#     endDate = datetime.datetime.strptime(request.args.get('end'), '%Y-%m-%d %H:%M:%S.%f')
#     with open('values.csv', mode='w') as csvFile:
        
        
    try:
        return send_file('values.csv', attachment_filename='values.csv')
    except Exception as e:
        return str(e)
    

if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2021 16:27:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2021 16:27:06] "GET /get HTTP/1.1" 200 -
